In [ ]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
from s3fs import S3FileSystem
import boto3
from botocore.exceptions import NoCredentialsError

In [ ]:
s3_client = boto3.client('s3', aws_access_key_id='<<aws_access_key_id>>', aws_secret_access_key='<<aws_secret_access_key>>')
bucket_name = 'kafka-cloud-comp-project'

In [ ]:
s3 = S3FileSystem()

In [ ]:
consumer = KafkaConsumer(
    'demo_test',
     bootstrap_servers=['<<Public IP>>'], #add your IP here
    value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
#pass values in producer and check here
for count, i in enumerate(consumer):
    print(count)
    print(i.value)

In [ ]:
# Stream data into S3 buckets
for count, message in enumerate(consumer):
    # Check if message.value is a dictionary
    if isinstance(message.value, dict):
        # It's already a dictionary, no need to decode
        json_data = message.value
    else:
        # If it's not a dictionary, decode and load it as JSON
        json_data = json.loads(message.value.decode('utf-8'))

    # Prepare the file key for S3
    file_key = f"stock_market_{count}.json"

    # Convert JSON data to string
    json_string = json.dumps(json_data)

    # Try to upload the file to S3
    try:
        s3_client.put_object(Body=json_string, Bucket=bucket_name, Key=file_key)
        print(f"Upload of {file_key} Successful")
    except NoCredentialsError:
        print("Credentials not available")